# Imports

In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report


# Class Node

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor for the Node class which is used to store the feature index, threshold, left and right child, information gain and value for leaf node. ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

# Class Decision Tree Classifier

In [3]:
class DecisionTreeClassifier():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor for the DecisionTreeClassifier class which initializes the root of the tree and stopping conditions, min_samples_split and max_depth. '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree by splitting the dataset until the stopping conditions are met ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split by looping over all the features and their possible thresholds, then computing the information gain and updating the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data based on the feature index and threshold '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        ''' function to print the tree '''
        
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        ''' function to fit the model '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)



# Loading and splitting the data

In [4]:
data = pd.read_csv('analysis/feature_list.csv')

X = data.iloc[:, :-1].values
Y = data.iloc[:, -1].values.reshape(-1,1)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=42)

# Fit classifier on data

In [5]:
classifier = DecisionTreeClassifier(min_samples_split=5, max_depth=30)
classifier.fit(X_train,Y_train)

# Evaluation

In [6]:
Y_pred = classifier.predict(X_test) 

#  Fast output of the classifier's accuracy

print('Accuracy Decision Tree:',accuracy_score(Y_test, Y_pred),'\n')
print('**********************************\n')

#***************************************************************
# Showing of the confusion matrix
# Predicting the classes with the test data set
y_pred = classifier.predict(X_test) #take test data as prediction data
print(confusion_matrix(Y_test, y_pred),'\n')
print('**********************************\n')

#***************************************************************
# Showing the classification report
print(classification_report(Y_test, y_pred))

Accuracy Decision Tree: 0.8674242424242424 

**********************************

[[67  0  0  3]
 [ 0 53 11  0]
 [ 0 18 42  2]
 [ 0  1  0 67]] 

**********************************

                    precision    recall  f1-score   support

            broken       1.00      0.96      0.98        70
closed_seal_broken       0.74      0.83      0.78        64
     closed_sealed       0.79      0.68      0.73        62
       open_broken       0.93      0.99      0.96        68

          accuracy                           0.87       264
         macro avg       0.86      0.86      0.86       264
      weighted avg       0.87      0.87      0.87       264



# Comparison: SKlearn

In [7]:
# importiere of sklearn classifier
from sklearn.tree import DecisionTreeClassifier
# Train and fit classifier on data
tree_clf = DecisionTreeClassifier()
tree_clf.fit(X_train,Y_train)

# Fast output of the classifier's accuracy
print('Accuracy Decision Tree:',tree_clf.score(X_test,Y_test),'\n')
print('**********************************\n')

# Showing of the confusion matrix
# Predicting the classes with the test data set
y_pred = tree_clf.predict(X_test) #take test data as prediction data
print(confusion_matrix(Y_test, y_pred),'\n')
print('**********************************\n')

#***************************************************************
# Showing the classification report
print(classification_report(Y_test, y_pred))

Accuracy Decision Tree: 0.8257575757575758 

**********************************

[[67  0  1  2]
 [ 0 44 20  0]
 [ 0 20 40  2]
 [ 0  1  0 67]] 

**********************************

                    precision    recall  f1-score   support

            broken       1.00      0.96      0.98        70
closed_seal_broken       0.68      0.69      0.68        64
     closed_sealed       0.66      0.65      0.65        62
       open_broken       0.94      0.99      0.96        68

          accuracy                           0.83       264
         macro avg       0.82      0.82      0.82       264
      weighted avg       0.83      0.83      0.83       264

